**Testing notebook for NoRep App** (Step by step)

In [1]:
# IMPORT PACKAGES

import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
mp_drawing = mp.solutions.drawing_utils   # This will give you all the drawing utilities
mp_pose = mp.solutions.pose               # Pose estimation model

In [ ]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):    # Press 'q' to close it
        break
        
cap.release()
cv2.destroyAllWindows()

In [2]:
# MAKE DETECTIONS

cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,30,0), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(187,225,160), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('VID_20220823_114722-Reduced.mp4')

lmrk = []

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            for lmk in landmarks:
                lmrk.append(lmk)
                print(lmk)
                
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,30,0), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(187,225,160), thickness=2, circle_radius=2)  
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [2]:
# CALCULATE ANGLES

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [ ]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
import pickle

with open('repcounter.p', 'rb') as f:
    model = pickle.load(f)

In [62]:
# CURL COUNTER

cap = cv2.VideoCapture(0)

# Counter variables
counter = 0 
grip = None
stance = None
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            # Grip
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            # Stance
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            
            # Calculate angles
            l_grip = calculate_angle(r_shoulder, l_shoulder, l_elbow)
            r_grip = calculate_angle(l_shoulder, r_shoulder, r_elbow)
            l_stance = calculate_angle(r_hip, l_hip, l_ankle)
            r_stance = calculate_angle(l_hip, r_hip, r_ankle)
            
            # Visualize angle  -  TAKE THIS PART OUT WHEN YOU FINISH!!
            cv2.putText(image, str(l_grip), 
                           tuple(np.multiply(l_shoulder, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(r_grip), 
                           tuple(np.multiply(r_shoulder, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )

            cv2.putText(image, str(l_stance), 
                           tuple(np.multiply(l_ankle, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(r_stance), 
                           tuple(np.multiply(r_ankle, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2, cv2.LINE_AA
                                )


            # Grip logic
            if (r_grip>90)|(l_grip>90)&(r_grip<120)|(l_grip<120):
                grip = 'Grip: Good!'
                posturebox = cv2.rectangle(image, (0,150), (225,73), (200,200,16), -1)
            if (r_grip>120)|(l_grip>120):
                grip = 'Grip: Too wide'
                posturebox = cv2.rectangle(image, (0,150), (225,73), (56,38,163), -1)
            if (r_grip<90)|(l_grip<90):
                grip = 'Grip: Too narrow'
                posturebox = cv2.rectangle(image, (0,150), (225,73), (56,38,163), -1)

            # Stance logic
            if (r_stance>88)|(l_stance>88)&(r_stance<98)|(l_stance<98):
                stance = 'Stance: Good!'
                posturebox = cv2.rectangle(image, (0,150), (225,73), (200,200,16), -1)
            if (r_stance>98)|(l_stance>98):
                stance = 'Stance: Too wide'
                posturebox = cv2.rectangle(image, (0,150), (225,73), (0,145,218), -1)
            if (r_stance<88)|(l_stance<88):
                stance = 'Stance: Too narrow'
                posturebox = cv2.rectangle(image, (0,150), (225,73), (0,145,218), -1)

            # Model implementation
            poses = results.pose_landmarks.landmark
            pose_row = np.array([[landmark.x, landmark.y, landmark.z] for landmark in poses]).flatten()
            frame_height, frame_width = frame.shape[:2]
            pose_row = pose_row * np.array([frame_width, frame_height, frame_width])[:,None]
            X = pd.DataFrame([pose_row[0]])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]

            str(body_language_class)

            # Rep counter logic
            if body_language_class == 0:
                stage = 'Down'
            if (body_language_class == 1)&(stage=='Down'):
                stage = 'Up'
                counter +=1
                                
        except:
            pass
        
        
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (87,122,59), -1)
        posturebox
        
        # Rep data
        cv2.putText(image, 'REPS', (25,15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (30,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Posture data
        cv2.putText(image, 'POSTURE', (70,90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, grip, (15,115), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(image, stance, (15,140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
        
        # Stage data

        cv2.putText(image, 'STAGE', (145,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (130,45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Display Probability
        cv2.putText(image, f'CONF:{str(round(body_language_prob[np.argmax(body_language_prob)],2))}', (130,68), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,30,0), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(187,225,160), thickness=2, circle_radius=2)  
                                 )               
        
        cv2.imshow('NoRep app', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

c:\Users\asgom\anaconda3\envs\norepenv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\asgom\anaconda3\envs\norepenv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\asgom\anaconda3\envs\norepenv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\asgom\anaconda3\envs\norepenv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\asgom\anaconda3\envs\norepenv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [13]:
str(body_language_class)

'0'

In [5]:
import pickle

with open('repcounter.p', 'rb') as f:
    model = pickle.load(f)

In [35]:
pose = results.pose_landmarks.landmark
pose_row = np.array([[landmark.x, landmark.y, landmark.z] for landmark in pose]).flatten()
frame_height, frame_width = frame.shape[:2]
pose_row = pose_row * np.array([frame_width, frame_height, frame_width])[:,None]
X = pd.DataFrame([pose_row[0]])
body_language_class = model.predict(X)[0]
body_language_prob = model.predict_proba(X)[0]
print(body_language_class, body_language_prob)

1 [0. 1.]


c:\Users\asgom\anaconda3\envs\norepenv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\asgom\anaconda3\envs\norepenv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [26]:
frame_height, frame_width = frame.shape[:2]
pose_row = pose_row * np.array([frame_width, frame_height, frame_width])[:,None]
list(pose_row)

[array([ 882594.53125   ,  100799.07226562, -598453.515625  ,
         895023.6328125 ,   81391.68701172, -572171.38671875,
         901905.76171875,   83146.19140625, -572136.08398438,
         907658.59375   ,   84622.78442383, -572129.58984375,
         876135.25390625,   79072.80273438, -569426.70898438,
         870638.671875  ,   79579.79125977, -569524.85351562,
         864693.26171875,   80202.48413086, -569568.65234375,
         915610.05859375,  103285.92529297, -401930.90820312,
         860262.59765625,  100231.18896484, -388882.54394531,
         894978.90625   ,  141234.31396484, -534553.95507812,
         870790.13671875,  137999.46289062, -531078.75976562,
         972989.2578125 ,  317369.87304688, -286985.3515625 ,
         814267.96875   ,  323223.65722656, -244573.58398438,
         988239.6484375 ,  552288.52539062, -209514.23339844,
         792679.58984375,  552885.15625   , -157159.15527344,
        1002860.64453125,  758462.5       , -318375.83007812,
        

In [15]:
frame_height
frame_width

1280